In [35]:
import os
import sys 
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K

from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.layers import Attention #from attention import Attention

import plotly
import plotly.graph_objects as go
import plotly.express as px
import kaleido ##pip install -U kaleido ##to save a plotly fig
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score,cross_val_predict


##https://keras.io/examples/timeseries/timeseries_transformer_classification/

In [36]:
print(f"Tensor Flow Version: {tf.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.8.0
GPU is available


In [37]:
ddf1 = pd.read_csv('Stable_Data_CSV.csv')
df2 = pd.read_csv('12w_features.csv')
df_stable = df2[df2.set_index(['participant_id']).index.isin(df1.set_index(['participant_id']).index)]
df_stable_US = df_stable.loc[(df_stable['arm'] == 1)]
df_stable_WN = df_stable.loc[(df_stable['arm'] == 2)]
print(len(df_stable_US))
print(len(df_stable_WN))

31
30


In [38]:
df_stable_US=(df_stable_US[['c_3','sl_1','a_2','q_1','tfi_total','c_3.1','sl_1.1','a_2.1','q_1.1','tfi_total.1','c_3.2','sl_1.2','a_2.2','q_1.2','tfi_total.2','tfi_total.3']])
df_stable_WN=(df_stable_WN[['sl_2','r_1','e_1','tfi_total','sl_2.1','r_1.1','e_1.1','tfi_total.1','sl_2.2','r_1.2','e_1.2','tfi_total.2','tfi_total.3']])

In [70]:
df_stable_WN

,sl_2,r_1,e_1,tfi_total,sl_2.1,r_1.1,e_1.1,tfi_total.1,sl_2.2,r_1.2,e_1.2,tfi_total.2,tfi_total.3
51,7.0,8.0,8.0,72.4,5.0,8.0,6.0,64.0,8.0,9.0,7.0,72.4,80.4
52,6.0,7.0,5.0,60.0,3.0,5.0,4.0,47.6,6.0,6.0,6.0,64.0,71.2
53,8.0,9.0,5.0,68.0,6.0,6.0,3.0,57.2,6.0,6.0,4.0,60.4,64.8
55,6.0,6.0,4.0,41.2,3.0,7.0,2.0,37.2,2.0,6.0,2.0,27.6,44.8
56,1.0,4.0,6.0,52.8,3.0,3.0,4.0,51.2,5.0,4.0,5.0,55.2,70.0
57,1.0,5.0,5.0,55.2,1.0,5.0,6.0,55.2,0.0,1.0,1.0,14.0,14.0
59,6.0,8.0,9.0,50.8,5.0,8.0,6.0,55.2,2.0,6.0,1.0,26.0,37.6
60,10.0,7.0,2.0,74.4,7.0,7.0,3.0,63.2,6.0,10.0,4.0,73.6,83.2
61,3.0,7.0,6.0,55.6,3.0,5.0,5.0,47.2,0.0,3.0,5.0,35.2,42.8
63,1.0,2.0,2.0,53.6,1.0,7.0,2.0,51.6,2.0,2.0,1.0,44.0,35.2


In [71]:
n_timesteps=3
n_features_US=5
n_features_WN=4

def data_prep(df,n_features):
    sc_X = StandardScaler()
    sc_y = StandardScaler()
    
    X=df.drop(['tfi_total.3'],axis=1)
    y=df[['tfi_total.3']]
    
    #scale x
    x_scaler=sc_X.fit(X)
    X=x_scaler.transform(X)
    #scale y
    y_scaler=sc_y.fit(y)
    y=y_scaler.transform(y)
    
    """
    Reshape rule:
    tensor of shape (batch size, sequence length, features), 
    where sequence length is the number of time steps and features is each input timeseries.
    """
    #X = X.reshape((X.shape[0], X.shape[1], 1))
    X = X.reshape((X.shape[0], n_timesteps, n_features))
    print(X.shape,y.shape)
    return X,y,x_scaler,y_scaler

In [72]:

"""
Include residual connections, layer normalization, and dropout.
The resulting layer can be stacked multiple times.
The projection layers are implemented through `keras.layers.Conv1D`.
"""

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Dense(ff_dim, activation="relu")(res)
    x = layers.Dropout(dropout)(x) 
    x = layers.Dense(units=inputs.shape[-1])(x) 
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res


def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    K.clear_session()
    inputs = keras.Input(shape=input_shape)
    x = inputs
    
    """    
    can stack multiple of the transformer_encoder blocks and 
    can also proceed to add the final Multi-Layer Perceptron regression head.
    """
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
    
    """
    a pooling layer is used to to reduce the output tensor of the TransformerEncoder 
    part of our model down to a vector of features for each data point in the current batch.
    """
    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x) 
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="linear")(x)
    return keras.Model(inputs, outputs)


In [73]:
## Train and evaluate


def prediction(x_train,y_train,y_scaler):
    input_shape = x_train.shape[1:]

    model = build_model(
        input_shape,
        head_size=5, # key_dim - Size of each attention head for query and key
        num_heads=4, # Number of attention heads
        ff_dim=5, # Hidden layer size in feed forward network inside transformer
        num_transformer_blocks=4,
        mlp_units=[128],
        mlp_dropout=0.4,
        dropout=0.25,
    )

    def error_in_tfi(y_true,y_pred): 
        y=y_true.numpy()
        yhat=y_pred.numpy()
        y=np.reshape(y, (1,-1))
        yhat=np.reshape(yhat, (1,-1))
        y=y_scaler.inverse_transform(y)
        yhat=y_scaler.inverse_transform(yhat)
        y=tf.convert_to_tensor(y, dtype=tf.float32)
        yhat=tf.convert_to_tensor(yhat, dtype=tf.float32)
        return K.mean(abs(y - yhat), axis=-1)  #K.mean(square(y_true - y_pred), axis=-1)


    model.compile(
        loss="mse",
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        #metrics=[keras.metrics.MeanAbsoluteError()],
        run_eagerly=True,
        metrics=[error_in_tfi],
    )
    model.summary()

    callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

    history=model.fit(
        x_train,
        y_train,
        validation_split=0.2,
        epochs=500,
        batch_size=4,
        callbacks=callbacks,
    )
    
    return model,history
    #model.evaluate(x_test, y_test, verbose=1)


In [74]:


error_scores_US=[]
error_scores_WN=[]
kfold = KFold(n_splits=5)


def cross_val(df,n_features,error_scores):
    X,y,x_scaler,y_scaler=data_prep(df,n_features)
    
    for train_index, test_index in kfold.split(X):
        print("TRAIN:", train_index, "TEST:", test_index)
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model,history=prediction(x_train,y_train,y_scaler)
        val_mse,val_mae=model.evaluate(x_test,y_test) #returns -- test loss, test metrics
        error_scores.append(val_mae)
    return history,x_scaler,y_scaler,error_scores

US_history,US_x_scaler,US_y_scaler,US_error_scores=cross_val(df_stable_US,n_features_US,error_scores_US)
WN_history,WN_x_scaler,WN_y_scaler,WN_error_scores=cross_val(df_stable_WN,n_features_WN,error_scores_WN)

(31, 3, 5) (31, 1)
TRAIN: [ 7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30] TEST: [0 1 2 3 4 5 6]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 5)]       0           []                               
                                                                                                  
 multi_head_attention (MultiHea  (None, 3, 5)        465         ['input_1[0][0]',                
 dAttention)                                                      'input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 3, 5)         0           ['multi_head_attention[0][0]']   
                                                                       

 tf.__operators__.add_6 (TFOpLa  (None, 3, 5)        0           ['layer_normalization_6[0][0]',  
 mbda)                                                            'tf.__operators__.add_5[0][0]'] 
                                                                                                  
 dense_6 (Dense)                (None, 3, 5)         30          ['tf.__operators__.add_6[0][0]'] 
                                                                                                  
 dropout_7 (Dropout)            (None, 3, 5)         0           ['dense_6[0][0]']                
                                                                                                  
 dense_7 (Dense)                (None, 3, 5)         30          ['dropout_7[0][0]']              
                                                                                                  
 layer_normalization_7 (LayerNo  (None, 3, 5)        10          ['dense_7[0][0]']                
 rmalizati

 dropout (Dropout)              (None, 3, 5)         0           ['multi_head_attention[0][0]']   
                                                                                                  
 layer_normalization (LayerNorm  (None, 3, 5)        10          ['dropout[0][0]']                
 alization)                                                                                       
                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 3, 5)        0           ['layer_normalization[0][0]',    
 da)                                                              'input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 3, 5)         30          ['tf.__operators__.add[0][0]']   
                                                                                                  
 dropout_1

 layer_normalization_7 (LayerNo  (None, 3, 5)        10          ['dense_7[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 tf.__operators__.add_7 (TFOpLa  (None, 3, 5)        0           ['layer_normalization_7[0][0]',  
 mbda)                                                            'tf.__operators__.add_6[0][0]'] 
                                                                                                  
 global_average_pooling1d (Glob  (None, 3)           0           ['tf.__operators__.add_7[0][0]'] 
 alAveragePooling1D)                                                                              
                                                                                                  
 dense_8 (Dense)                (None, 128)          512         ['global_average_pooling1d[0][0]'
          

 input_1 (InputLayer)           [(None, 3, 5)]       0           []                               
                                                                                                  
 multi_head_attention (MultiHea  (None, 3, 5)        465         ['input_1[0][0]',                
 dAttention)                                                      'input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 3, 5)         0           ['multi_head_attention[0][0]']   
                                                                                                  
 layer_normalization (LayerNorm  (None, 3, 5)        10          ['dropout[0][0]']                
 alization)                                                                                       
                                                                                                  
 tf.__oper

...

Total params: 2,437
Trainable params: 2,437
Non-trainable params: 0
__________________________________________________________________________________________________
Epoch 1/500
5/5 [==============================] - 1s 166ms/step - loss: 1.1415 - error_in_tfi: 19.3475 - val_loss: 0.0445 - val_error_in_tfi: 2.9546
Epoch 2/500
5/5 [==============================] - 1s 141ms/step - loss: 1.0365 - error_in_tfi: 18.5211 - val_loss: 0.0383 - val_error_in_tfi: 3.3378
Epoch 3/500
5/5 [==============================] - 1s 137ms/step - loss: 0.9905 - error_in_tfi: 17.7199 - val_loss: 0.0386 - val_error_in_tfi: 3.8095
Epoch 4/500
5/5 [==============================] - 1s 135ms/step - loss: 0.8844 - error_in_tfi: 16.7837 - val_loss: 0.0403 - val_error_in_tfi: 4.0482
Epoch 5/500
5/5 [==============================] - 1s 133ms/step - loss: 0.8986 - error_in_tfi: 16.3104 - val_loss: 0.0470 - val_error_in_tfi: 4.6958
Epoch 6/500
5/5 [==============================] - 1s 133ms/step - loss: 0.8380 - e

 mbda)                                                            'tf.__operators__.add_3[0][0]'] 
                                                                                                  
 dense_4 (Dense)                (None, 3, 5)         25          ['tf.__operators__.add_4[0][0]'] 
                                                                                                  
 dropout_5 (Dropout)            (None, 3, 5)         0           ['dense_4[0][0]']                
                                                                                                  
 dense_5 (Dense)                (None, 3, 4)         24          ['dropout_5[0][0]']              
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 3, 4)        8           ['dense_5[0][0]']                
 rmalization)                                                                                     
          

 dAttention)                                                      'input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 3, 4)         0           ['multi_head_attention[0][0]']   
                                                                                                  
 layer_normalization (LayerNorm  (None, 3, 4)        8           ['dropout[0][0]']                
 alization)                                                                                       
                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 3, 4)        0           ['layer_normalization[0][0]',    
 da)                                                              'input_1[0][0]']                
                                                                                                  
 dense (De

 dense_7 (Dense)                (None, 3, 4)         24          ['dropout_7[0][0]']              
                                                                                                  
 layer_normalization_7 (LayerNo  (None, 3, 4)        8           ['dense_7[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 tf.__operators__.add_7 (TFOpLa  (None, 3, 4)        0           ['layer_normalization_7[0][0]',  
 mbda)                                                            'tf.__operators__.add_6[0][0]'] 
                                                                                                  
 global_average_pooling1d (Glob  (None, 3)           0           ['tf.__operators__.add_7[0][0]'] 
 alAveragePooling1D)                                                                              
          

Epoch 42/500
5/5 [==============================] - 1s 141ms/step - loss: 0.2544 - error_in_tfi: 7.3167 - val_loss: 0.4460 - val_error_in_tfi: 8.4658
Epoch 43/500
5/5 [==============================] - 1s 142ms/step - loss: 0.3788 - error_in_tfi: 9.4413 - val_loss: 0.4397 - val_error_in_tfi: 8.1749
Epoch 44/500
5/5 [==============================] - 1s 140ms/step - loss: 0.2783 - error_in_tfi: 7.6761 - val_loss: 0.4369 - val_error_in_tfi: 8.1344
Epoch 45/500
5/5 [==============================] - 1s 140ms/step - loss: 0.3152 - error_in_tfi: 8.4026 - val_loss: 0.4354 - val_error_in_tfi: 8.0432
Epoch 46/500
5/5 [==============================] - 1s 135ms/step - loss: 0.3068 - error_in_tfi: 8.2963 - val_loss: 0.4440 - val_error_in_tfi: 8.1484
Epoch 47/500
5/5 [==============================] - 1s 134ms/step - loss: 0.2335 - error_in_tfi: 6.7997 - val_loss: 0.4473 - val_error_in_tfi: 8.1592
Epoch 48/500
5/5 [==============================] - 1s 136ms/step - loss: 0.2625 - error_in_tfi: 7.3

In [77]:
print("US_error")
print("--------")
print(US_error_scores)
US_error=np.mean(US_error_scores)
print("\n")
print(US_error)
print("\n")
print("WN_error")
print("--------")
print(WN_error_scores)
WN_error=np.mean(WN_error_scores)
print("\n")
print(WN_error)

US_error
--------
[10.833792686462402, 12.296378135681152, 15.793066024780273, 12.977690696716309, 9.746867179870605]


12.329558944702148


WN_error
--------
[20.627243041992188, 9.532112121582031, 11.078472137451172, 23.999364852905273, 13.714899063110352]


15.790418243408203
